# Trigram Language Model

In [1]:
# Imports
import json
import random
import math
from collections import defaultdict, Counter
import sys
sys.path.append("../tokenizer")
from bpe_tokenizer import get_tokenizer, load_tokenizer, EOT, EOS, EOP

In [2]:
with open("../tokenizer/tokenized_corpus.json", "r", encoding="utf-8") as f:
    corpus = json.load(f)

merges, char2id, id2char = load_tokenizer("../tokenizer")

EOT_ID = char2id[EOT]
VOCAB_SIZE = 250

tokenized_stories = [story["tokens"] for story in corpus]

print(f"Loaded {len(tokenized_stories)} stories, Vocab size: {VOCAB_SIZE}, EOT_ID: {EOT_ID}")

Loaded 500 stories, Vocab size: 250, EOT_ID: 0


In [3]:
# Trigram Language Model with Interpolation and Perplexity Calculation
class TrigramLanguageModel:
    def __init__(self, vocab_size):
        self.unigrams = Counter()
        self.bigrams = Counter()
        self.trigrams = Counter()
        self.total_tokens = 0
        self.vocab_size = vocab_size

    def train(self, tokenized_stories):
        for story in tokenized_stories:
            self.total_tokens += len(story)

            for i in range(len(story)):
                self.unigrams[story[i]] += 1

                if i >= 1:
                    self.bigrams[(story[i-1], story[i])] += 1

                if i >= 2:
                    self.trigrams[(story[i-2], story[i-1], story[i])] += 1

    def unigram_prob(self, w):
        return self.unigrams[w] / self.total_tokens

    def bigram_prob(self, w1, w2):
        if self.unigrams[w1] == 0:
            return 0
        return self.bigrams[(w1, w2)] / self.unigrams[w1]

    def trigram_prob(self, w1, w2, w3):
        if self.bigrams[(w1, w2)] == 0:
            return 0
        return self.trigrams[(w1, w2, w3)] / self.bigrams[(w1, w2)]

    def interpolated_prob(self, w1, w2, w3, l1, l2, l3):
        p1 = self.unigram_prob(w3)
        p2 = self.bigram_prob(w2, w3)
        p3 = self.trigram_prob(w1, w2, w3)
        return l1 * p1 + l2 * p2 + l3 * p3

    def perplexity(self, tokenized_stories, l1, l2, l3):
        log_prob_sum = 0
        N = 0

        for story in tokenized_stories:
            for i in range(2, len(story)):
                w1, w2, w3 = story[i-2], story[i-1], story[i]
                prob = self.interpolated_prob(w1, w2, w3, l1, l2, l3)

                if prob > 0:
                    log_prob_sum += math.log(prob)
                else:
                    log_prob_sum += math.log(1e-10)

                N += 1

        return math.exp(-log_prob_sum / N)

    def generate(self, prefix_tokens, l1, l2, l3, eot_id):
        tokens = list(prefix_tokens)

        while len(tokens) < 2:
            tokens.insert(0, tokens[0] if tokens else 0)

        while True:
            w1, w2 = tokens[-2], tokens[-1]

            probs = []
            for token_id in range(self.vocab_size):
                p = self.interpolated_prob(w1, w2, token_id, l1, l2, l3)
                probs.append(p)

            next_token = random.choices(range(self.vocab_size), weights=probs, k=1)[0]
            tokens.append(next_token)

            if next_token == eot_id:
                break

        return tokens

In [4]:
def tune_lambdas(model, dev_data):
    best_perplexity = float("inf")
    best_lambdas = (0, 0, 0)

    for l1 in [0.1, 0.2, 0.3]:
        for l2 in [0.1, 0.2, 0.3]:
            l3 = 1 - l1 - l2
            if l3 <= 0:
                continue

            perp = model.perplexity(dev_data, l1, l2, l3)

            if perp < best_perplexity:
                best_perplexity = perp
                best_lambdas = (l1, l2, l3)

    return best_lambdas


def split_data(tokenized_stories):
    shuffled = tokenized_stories.copy()
    random.shuffle(shuffled)
    n = len(shuffled)

    train = shuffled[:int(0.7*n)]
    dev   = shuffled[int(0.7*n):int(0.8*n)]
    test  = shuffled[int(0.8*n):]

    return train, dev, test

In [5]:
train_data, dev_data, test_data = split_data(tokenized_stories)

model = TrigramLanguageModel(VOCAB_SIZE)
model.train(train_data)

print(f"Train: {len(train_data)}, Dev: {len(dev_data)}, Test: {len(test_data)}")
print(f"Total tokens: {model.total_tokens:,}")

Train: 350, Dev: 50, Test: 100
Total tokens: 528,613


In [6]:
l1, l2, l3 = tune_lambdas(model, dev_data)
print(f"Best lambdas: l1={l1}, l2={l2}, l3={l3}")

test_perplexity = model.perplexity(test_data, l1, l2, l3)
print(f"Test Perplexity: {test_perplexity:.2f}")

Best lambdas: l1=0.1, l2=0.2, l3=0.7
Test Perplexity: 19.60


In [7]:
final_model = TrigramLanguageModel(VOCAB_SIZE)
final_model.train(tokenized_stories)
print(f"Final model trained on {final_model.total_tokens:,} tokens")

Final model trained on 766,407 tokens


In [8]:
model_data = {
    "unigrams": dict(final_model.unigrams),
    "bigrams": {str(k): v for k, v in final_model.bigrams.items()},
    "trigrams": {str(k): v for k, v in final_model.trigrams.items()},
    "total_tokens": final_model.total_tokens,
    "vocab_size": final_model.vocab_size,
    "lambdas": [l1, l2, l3],
    "eot_id": EOT_ID
}

with open("trigram_model.json", "w", encoding="utf-8") as f:
    json.dump(model_data, f)

print("Model saved to trigram_model.json")

Model saved to trigram_model.json


In [9]:
tokenizer = get_tokenizer("../tokenizer")

In [10]:
prefix = "ایک دفعہ کا ذکر ہے"
prefix_tokens = tokenizer.encode(prefix)


generated_tokens = final_model.generate(prefix_tokens,l1, l2, l3, EOT_ID)
print(tokenizer.decode(generated_tokens))

ایک دفعہ کا ذکر ہے، مان ہو گیا۔ ␞اگر سب،بہت ع کا منہ چل پڑیں گے۔ ␞کباب بھی بے وں زینک گنتی بچپند ہفتے بیٹر گلک شکیلے تھے" جنہوں اور کہا:"بھر پڑا رہے تھے۔ ␞ تھ دو تینوہوں؟ ␞لگتا ات کریں یہی ہے  کامجد ہو گئے۔ ␞ ␝ حاو بھائاپنی ع"سائیکلن سے کایا تو اس کے نزاری سے کہا۔ ␞کا تھا۔ ␞ ␝ "تمہارے بہا گمجھرارتے۔ ␞ ␝ غصے سے آہپسننے کے نزدرات ہے نام ہو گیا۔ وقاُٹھا کر مکان سے مل کوؤ۔ ␞"دادریس جگہ پر پکڑاینلز اِدھر سے کروازر میں کھڑا تو تمام ابھی اس ل کی ہے۔ ␞ ␝ حلے کر آ کر انھیں اسکٹا! ␞ ␝ کرشوطا نظر آ جادو ہانیہ اس کی دادا جبنا لیں۔ ␞اپنے والے میں کیا تو چوہے میں بھی منے پھر اس نے میٹھا لی اور اس کی آنکھوں سے چھ نظر دیکھا اس کی نیک کلو شر پڑ ␝ رضا انٹی کا وہاں انپ سے اتولاد کے پیچھے بعد بر کیے۔ ␞ مجھے لے ہی اس کی اس لئے انہوعافسیا␝ "اسے بنگسے سہیت بھی کا کیا۔ ␞مجھے بہت سن کر سونگھ  کے پہ د سونے کا انا آ گچارپائی نے تو بیف کرکے پینے اٹ کر دوں گا۔ ␞اس پت،گھر کرتا ہے۔ ␞آپ گئیں۔ ␞گوش کی تو تمبیما کر رونے لگے گاور وہ لوگوںرہا تھا:"کیا اور کال۔ ␞ ␝ ارسپر منور کو آواز․․․․؟ ␞یہ تو بہت سادرو␞اور برا نھوں نے 

In [11]:
# loading model from the file
with open("trigram_model.json", "r", encoding="utf-8") as f:
    loaded = json.load(f)

# Reconstruct model
test_model = TrigramLanguageModel(loaded["vocab_size"])
test_model.unigrams = Counter(loaded["unigrams"])
test_model.bigrams = Counter({eval(k): v for k, v in loaded["bigrams"].items()})
test_model.trigrams = Counter({eval(k): v for k, v in loaded["trigrams"].items()})
test_model.total_tokens = loaded["total_tokens"]

l1, l2, l3 = loaded["lambdas"]
eot_id = loaded["eot_id"]

# Generate until EOT
prefix = "ایک دفعہ کا ذکر ہے"
prefix_tokens = tokenizer.encode(prefix)
generated_tokens = test_model.generate(prefix_tokens, l1, l2, l3, eot_id)
generated_text = tokenizer.decode(generated_tokens)

print(f"Generated {len(generated_tokens)} tokens")
print(generated_text)

Generated 1500 tokens
ایک دفعہ کا ذکر ہے، یہ آدمی کے ساتھ آرام فیصل کر حیرے کے پتا چلا جان بچانا نماز نہیں ہوا۔ ␞" سارے پیر جی سوکھی ابو کو کھا لڑکا دیئے۔ ␞ ␝ "جو بھی اپنی خواہ تو رنگ خوشی ہے کی کامیات کے قدرے فیصل کرا دروازے سے دورہ دیر تصویل پر رشیراز نے جمایا نہ پڑے۔ ␞اب ایس ہزادہ جاتے۔ ␞" ␝ اس کے کت کرنے میں اور آپس میں رہنے لگا تو آپ سال ۔ ␞" ␝ امی اٹھا لیا تو احمد کے قصاب نے آگے تم سے باہر پھول،رد ہونے والے تھے کہ "بی مغربان بنی اور اب کے سارہ بن  صحت بھلا کر بتا دی۔ ␞ ␝ منیر خوشیاء کی یہ بٹونا ٹوٹ کر بھی وہ نظار کرنا چاہ رہتی۔ ␞ ␝ وہ دونوں کو روکنے کی ایک تجرے کے لئے جیونجرہ چلانے کے لئے باہر نکلا۔ ␞جل پگھل مند نہ کر سکتا۔ ␞اس کے بابا نے اور کپڑے کو پیپر آئیں۔ ␞میں نے پین سفر پر ایک کھلے دو چھوٹی سی معنی ہانیہ بیگ کا شکارڈ لڑکی ہے کہ شاہ کی نئی ورزیر پر ڈالی وجہ بنیا کو ڈرایا تو سب کو سہاریہ کہہ کر کے دنوں اور سمان پر حمت،مگر مرغا صاحب نے پیار ہو گیا ہے،جبکرین قصائمہ پر لے جاؤ میری چھوٹس سے کہا شوق نوجوان کے والد نے ان کا قات پڑھنا پڑے:"شاباش دنیا بھی جاتا تھا ناشت کرنے سے گناہ 